# Mount Drive

In [1]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

# Install packages


In [2]:
# !pip install -q transformers
# # !pip install -q tensorflow==2.2-rc1
!pip install -q tf-models-official==2.2.0

In [3]:
# !pip install keras-lr-multiplier

# Import libraries

In [4]:
import os
import time
import datetime
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from transformers import TFAutoModel, AutoTokenizer, TFBertForSequenceClassification,AutoConfig
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Maximum,LayerNormalization,GlobalMaxPooling2D,Average,Dot, Dense, Input, GlobalAveragePooling1D, BatchNormalization, Activation, Concatenate, Flatten, Dropout, Conv1D, MaxPooling1D, Add, Lambda, GlobalAveragePooling2D, Reshape, RepeatVector, UpSampling1D 
from tensorflow.keras.models import Model
from keras.layers import LSTM, Bidirectional
from official import nlp
import official.nlp.optimization

2022-09-28 10:19:19.547579: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
tf.keras.backend.clear_session()

In [6]:
# Set seed value
seed_value = 56
os.environ['PYTHONHASHSEED']=str(0)
# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)
# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)
# 4. Set `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)
# 5. Configure a new global `tensorflow` session
from keras import backend as K
# session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
# K.set_session(sess)
# for later versions:
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)


2022-09-28 10:19:26.264047: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Parameters

In [7]:
base_dir    = '/Users/mryu/Desktop'
train_path  = os.path.join(base_dir, 'train.csv')
val_path    = os.path.join(base_dir, 'dev.csv')
test_path    = os.path.join(base_dir, 'test.csv')
img_train_path = os.path.join(base_dir, 'train_img.npy')
img_val_path = os.path.join(base_dir, 'dev_img.npy')
img_test_path = os.path.join(base_dir, 'test_img.npy')
MAX_LENGTH  = 32
MODEL       = 'vinai/bertweet-base'
MODEL_NAME  = 'vinai/bertweet-base'
N_LABELS    = 1

# Read data

In [8]:
df_train = pd.read_csv(train_path)
print(df_train.shape)
print(df_train.info())

display(df_train.head())

(5445, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5445 entries, 0 to 5444
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5445 non-null   int64 
 1   idx         5445 non-null   int64 
 2   clean_text  5445 non-null   object
 3   img_id      5445 non-null   object
 4   label       5445 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 212.8+ KB
None


Unnamed: 0   idx                                         clean_text  \
0        3997  4162  ben affleck and lindsay shookus are still grou...   
1        4641  4835  the average length of pregnancy is 40 weeks.  ...   
2         162   170  for all the rumors out there that jamie foxx a...   
3        2027  2118  why i love these comeback classics: some of ed...   
4        6204  6448  there's a third character in the friendship ta...   

                 img_id  label  
0      gossipcop-914193      0  
1      gossipcop-914670      0  
2  gossipcop-4769457532      1  
3      gossipcop-915701      0  
4      gossipcop-903697      0

In [9]:
df_val = pd.read_csv(val_path)
print(df_val.shape)
print(df_val.info())

display(df_val.head())

(683, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683 entries, 0 to 682
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  683 non-null    int64 
 1   idx         683 non-null    int64 
 2   clean_text  683 non-null    object
 3   img_id      683 non-null    object
 4   label       683 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 26.8+ KB
None


Unnamed: 0   idx                                         clean_text  \
0        1490  1564  gwen stefani is getting candid about what life...   
1         530   555  new you can now listen to fox news articles!  ...   
2        2670  2783  she played lois lane in the films in the 1970s...   
3        2609  2722  comedians seth rogen and billy eichner have si...   
4        6718  6975  just as important as the met gala theme reveal...   

                 img_id  label  
0  gossipcop-5656473493      1  
1  gossipcop-5483599093      1  
2      gossipcop-935084      0  
3      gossipcop-845984      0  
4      gossipcop-892358      0

In [10]:
df_test = pd.read_csv(test_path)
print(df_test.shape)
print(df_test.info())

display(df_test.head())

(677, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 677 entries, 0 to 676
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  677 non-null    int64 
 1   idx         677 non-null    int64 
 2   clean_text  677 non-null    object
 3   img_id      677 non-null    object
 4   label       677 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 26.6+ KB
None


Unnamed: 0   idx                                         clean_text  \
0        3733  3886  "i had a close friend of mine that didn't supp...   
1         796   832  const fp = {featured_posts_nonce:c24743471e,fe...   
2         519   544  kaley cuoco and big bang theory co-star mayim ...   
3         508   533  the death of owen wilsons beloved dad has frie...   
4        3259  3398  after suffering a miscarriage and struggling w...   

                 img_id  label  
0      gossipcop-855596      0  
1   gossipcop-178000221      1  
2  gossipcop-6038714010      1  
3  gossipcop-3222492495      1  
4      gossipcop-930238      0

In [11]:
# Get the lists of sentences and their labels.
train_sent      = df_train.clean_text.values
train_labels    = df_train.label.values
val_sent        = df_val.clean_text.values
val_labels      = df_val.label.values 
test_sent       = df_test.clean_text.values
test_labels     = df_test.label.values 

In [12]:
#Bertweet tokens
import re

for i in range(train_sent.shape[0]):
    train_sent[i] = re.sub(r'<url>','HTTPURL',train_sent[i])
    train_sent[i] = re.sub(r'<user>','@USER',train_sent[i])
for i in range(val_sent.shape[0]):
    val_sent[i] = re.sub(r'<url>','HTTPURL',val_sent[i])
    val_sent[i] = re.sub(r'<user>','@USER',val_sent[i])
for i in range(test_sent.shape[0]):
    test_sent[i] = re.sub(r'<url>','HTTPURL',test_sent[i])
    test_sent[i] = re.sub(r'<user>','@USER',test_sent[i])

In [13]:
train_sent   = np.append(train_sent,val_sent)
train_labels = np.append(train_labels,val_labels)
train_labels

array([0, 0, 1, ..., 1, 0, 0])

In [14]:
#Image
img_train = np.load(img_train_path)
img_val   = np.load(img_val_path)
img_test  = np.load(img_test_path)
img_train.shape,  img_test.shape, img_val.shape

((5445, 224, 224, 3), (677, 224, 224, 3), (683, 224, 224, 3))

In [15]:
#Use both train and val to train like other papers
img_train = np.vstack((img_train,img_val))
img_train.shape

(6128, 224, 224, 3)

In [16]:
print(len(train_sent), len(train_labels))
print(len(val_sent), len(val_labels))
print(len(test_sent), len(test_labels))

6128 6128
683 683
677 677


# Tokenization & Input Formatting


In [17]:
!pip install emoji

In [18]:
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = AutoTokenizer.from_pretrained(MODEL, do_lower_case=False)

Loading BERT tokenizer...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
print(' Original: ', train_sent[4])
print('Tokenized: ', tokenizer.tokenize(train_sent[4]))
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(train_sent[4])))

 Original:  there's a third character in the friendship tale of jennifer lawrence and emma stone: john the orchestra guy.  who is john? an orchestra guy, that's who -- an orchestra guy who used to constantly, "accidentally" text both stone and lawrence.  "he wasn't really a stalker. he's just this guy who has a lot of people's phone numbers," stone said in an interview for w's best performances feature. "he must have worked at a studio or something."  according to stone, john would often text both women to inform them he was running late to the "soundstage." he'd then asked them to "warm up the orchestra" for him. (now you see where the "orchestra" part of the name comes from!)  weird, right? but no harm done.  an issue arose, however, when lawrence and stone -- who had never met in person, nor talked on the phone -- decided to finally hang out irl. as lawrence drove to stone's house, a fear emerged. what if this wasn't emma after all, but a trap from john the orchestra guy? what if th

## Find max length

In [20]:
max_len = 0
stat = []
for sent in tqdm(train_sent):
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
#    if(len(input_ids)>96):
#      print(sent)
    max_len = max(max_len, len(input_ids))
    stat.append(len(input_ids))

print('\nMax sentence length in train data: ', max_len)

# for sent in tqdm(val_sent):
#     input_ids   = tokenizer.encode(sent, add_special_tokens=True)
#  #   if(len(input_ids)>96):
#  #     print(sent)
#     max_len     = max(max_len, len(input_ids))
#     stat.append(len(input_ids))

# print('\nMax sentence length in both train and val data: ', max_len)

for index, sent in tqdm(enumerate(test_sent)):
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    if (len(input_ids) > 96):
        print(index, sent)
    max_len = max(max_len, len(input_ids))
    stat.append(len(input_ids))

print('\nMax sentence length in both train and val data: ', max_len)

100%|██████████████████████████████████████| 6128/6128 [00:28<00:00, 216.29it/s]



Max sentence length in train data:  22948


29it [00:00, 287.88it/s]

0 "i had a close friend of mine that didn't support my album, but then, like, supported other people's stuff, and i asked him why," drake said during his acceptance speech after winning the award for top billboard 200 album for views at the billboard music awards. "the response was 'i don't want to look thirsty, and you get enough love as it is.'" that interaction taught drizzy a valuable lesson: "we're all here on earth for a limited amount of time. we gotta show love while we're here."  then, drake got a couple of things off his chest. "vanessa hudgens, you look...incredible tonight." now we all know what was going through the rapper's mind when the camera caught him staring at hudgens during the opening monologue.  nicki minaj, however, wasn't happy with drake's comment.  drake admitted that he and ludacris haven't always seen eye-to-eye, but he's still a big fan and loves him. drizzy also expressed his gratitude for finding his way back to nicki minaj.  related stories story contin

58it [00:00, 201.26it/s]

...

540it [00:02, 166.91it/s]

501 bella thorne: latest news, views, gossip, photos and video  bella thorne shares a passionate kiss with a handsome mystery man in mykonos - two months after shock split from fianc benjamin mascolo bella thorne was spotted packing on the pda with a handsome mystery man in mykonos, greece on saturday. the actress, 21, cosied up to the man before the pair shared a passionate kiss while soaking up the sun on a lavish yacht. it comes just two months after her fianc benjamin mascolo announced the pair had amicably split after a three year relationship, in a lengthy statement on the social media platform....read bella thorne packs on the pda with a handsome mystery man  bella thorne's ex tana mongeau wears a very low-cut outfit at the premiere of brad pitt's new action film bullet train in los angeles the 24-year-old ex of bella thorne wore a plunging black outfit that made it clear she was not in a bra as she wore her long hair over her shoulders. bella thorne's ex tana mongeau wears low-

580it [00:03, 163.69it/s]

551 american actress  sarah michelle prinze[2] (ne gellar ghel-r; born april 14, 1977[3]) is an american actress, producer, and entrepreneur. after being spotted at the age of four in new york city, she made her screen acting debut in the television film an invasion of privacy (1983). a leading role on the teen drama series swans crossing (1992) was followed by her role as kendall hart on the abc daytime soap opera all my children (19931995), for which she won the daytime emmy award for outstanding younger actress in a drama series.  gellar received international recognition for her portrayal of buffy summers on the wb/upn television series buffy the vampire slayer (19972003), which earned her five teen choice awards, a saturn award, and a golden globe award nomination.[4][5] in film, her most commercially successful performances include i know what you did last summer (1997), scream 2 (1997), cruel intentions (1999), scooby-doo (2002) and scooby-doo 2: monsters unleashed (2004), the g

644it [00:03, 229.12it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Tokenize 

### Train

In [21]:
input_ids       = []
attention_masks = []

for sent in tqdm(train_sent):
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, 
                        max_length = MAX_LENGTH,         
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'np',   
                        truncation = True,
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])


id_train        = np.concatenate(input_ids)
mask_train      = np.concatenate(attention_masks)
y_train         = train_labels 
id_train.shape, mask_train.shape, y_train.shape

  0%|                                                  | 0/6128 [00:00<?, ?it/s]/Users/mryu/opt/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2323: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████████████████████████████████| 6128/6128 [00:19<00:00, 316.43it/s]


((6128, 32), (6128, 32), (6128,))

### Val

In [22]:
input_ids       = []
attention_masks = []

for sent in tqdm(val_sent):
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, 
                        max_length = MAX_LENGTH,         
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'np',   
                        truncation = True,
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])


id_val          = np.concatenate(input_ids)
mask_val        = np.concatenate(attention_masks)
y_val           = val_labels
id_val.shape, mask_val.shape, y_val.shape

100%|████████████████████████████████████████| 683/683 [00:01<00:00, 353.74it/s]


((683, 32), (683, 32), (683,))

## Create iterator for data

In [23]:
BATCH_SIZE      = 256 

X_train         = [
    id_train,
    mask_train,
    img_train
]
X_val           = [
    id_val,
    mask_val,
    img_val  
]

# Train TFBertForSequenceClassification Model

In [24]:
# Change here for different models
from tensorflow.keras.applications import VGG19
# Change here for different models
from tensorflow.keras.applications.vgg19 import preprocess_input


def create_cnn(input_shape):
    inputs = Input(shape=input_shape)
    x = Lambda(preprocess_input)(inputs)
  # load the VGG16 network, ensuring the head FC layer sets are left
  # off                                                                           - Change here for different models
    baseModel = VGG19(weights="imagenet", include_top=True, input_tensor=x)

  # construct the head of the model that will be placed on top of the
  # the base model
    headModel_cnn = baseModel.layers[22].output
    headModel_fc = baseModel.layers[25].output
  # Attention
    headModel_cnn = Reshape((-1, headModel_cnn.shape[-1]))(headModel_cnn)

  # Non-attention
  # headModel = Flatten()(headModel)
  # headModel = Dense(512)(headModel)
  # headModel = BatchNormalization()(headModel)
  # headModel = Activation("relu")(headModel)
  # headModel = Dropout(0.2)(headModel)

  # headModel = Dense(512)(headModel)
  # headModel = BatchNormalization()(headModel)
  # headModel = Activation("relu")(headModel)
  # headModel = Dropout(0.2)(headModel)

    model = Model(inputs=baseModel.input, outputs=[
                  headModel_cnn, headModel_fc])

  # loop over all layers in the base model and freeze them so they will
  # *not* be updated during the first training process
    for layer in baseModel.layers:
        layer.trainable = False

    return model

In [25]:
def create_model(transformer, max_len=256):
    merge = []

    input_ids = Input(shape=(max_len, ), dtype=tf.int32, name='input_ids')
    attention_mask = Input(shape=(max_len, ),
                           dtype=tf.int32,
                           name='attention_mask')
    image = create_cnn((224, 224, 3))
    sequence_output_1 = transformer(input_ids,
                                    attention_mask=attention_mask)[2][-1]
    sequence_output_2 = transformer(input_ids,
                                    attention_mask=attention_mask)[2][-2]
    sequence_output_3 = transformer(input_ids,
                                    attention_mask=attention_mask)[2][-3]
    sequence_output_4 = transformer(input_ids,
                                    attention_mask=attention_mask)[2][-4]
    sequence_output = Concatenate(axis=2)([
        sequence_output_1, sequence_output_2, sequence_output_3,
        sequence_output_4
    ])

    #cls_token           = sequence_output[:, 0, :]

    #merge.append(cls_token)

    # Yoon Kim model (https://arxiv.org/abs/1408.5882)
    convs = []
    filter_sizes = [2, 3, 4, 5]
    size_pool = 3
    drop_rate = 0.2
    final_hid = 32

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=768, kernel_size=filter_size)(sequence_output)
        l_conv = BatchNormalization()(l_conv)
        l_conv = Activation('relu')(l_conv)
        # l_pool = MaxPooling1D(pool_size=max_len-filter_size+1)(l_conv)
        l_pool = MaxPooling1D(pool_size=size_pool)(l_conv)
        convs.append(l_pool)
        #merge.append(Flatten()(l_pool))

    l2_pool = Concatenate(axis=1)(convs)
    # l2_pool = BatchNormalization()(l2_pool)
    for _ in range(2):
        origin = l2_pool
        l2_conv = Conv1D(filters=768, kernel_size=size_pool,
                         padding='same')(l2_pool)
        l2_conv = BatchNormalization()(l2_conv)
        l2_conv = Activation('relu')(l2_conv)
        #print(origin.shape, l2_conv.shape)
        # l2_conv = Add()([Lambda(lambda x: x[0]*x[1])([origin,0.1]), l2_conv])
        l2_pool = MaxPooling1D(pool_size=size_pool)(l2_conv)

    text = Flatten()(l2_pool)
    #append to merge
    text_append = Dense(512)(text)
    text_append = BatchNormalization()(text_append)
    text_append = Activation('relu')(text_append)
    text_append = Dropout(drop_rate)(text_append)
    text_append = Dense(final_hid)(text_append)
    text_append = BatchNormalization()(text_append)
    text_append = Activation('relu')(text_append)

    # merge.append(text_append)
    # text_append = l2_pool

    #text for attention
    # text = Dropout(0.2)(text)
    # text = Dense(128)(text)
    # text = BatchNormalization()(text)
    # text = Activation("tanh")(text)                  #change to tanh?
    # text = Dropout(0.2)(text)
    image_append = Dense(2048)(image.output[1])
    image_append = BatchNormalization()(image_append)
    image_append = Activation('relu')(image_append)
    image_append = Dropout(drop_rate)(image_append)
    image_append = Dense(final_hid)(image_append)
    image_append = BatchNormalization()(image_append)
    image_append = Activation('relu')(image_append)

    img = Dense(final_hid)(image.output[0])
    img = BatchNormalization()(img)
    img = Activation('relu')(img)

    # image_append = image.output[1]
    # merge.append(image_append)
    # image_append = image.output[0]

    head = 1
    att_layers = 1
    att_hid = 32
    ##With attention - text on image
    inpAttImg_key = img
    inpAttImg_query = text_append
    for layer in range(1):
        att_img = []
        concat_key = []
        for _ in range(head):
            img_key = Dense(att_hid / head,
                            use_bias=False)(inpAttImg_key)  #change to tanh?
            text_query = Dense(att_hid / head, use_bias=False)(inpAttImg_query)
            # img_key = BatchNormalization()(img_key)
            # img_key = Activation('tanh')(img_key)
            # text_query = BatchNormalization()(text_query)
            # text_query = Activation('tanh')(text_query)

            img_value = Dense(att_hid / head, use_bias=False)(inpAttImg_key)
            # img_value = BatchNormalization()(img_value)
            # img_value = Activation('tanh')(img_value)

            attention = Dot(axes=(1, 2))([text_query, img_key])
            attention = Lambda(lambda x: x[0] / x[1])(
                [attention, np.sqrt(att_hid / head)])
            attention = Activation("softmax")(attention)
            head_att_img = Dot(axes=(1, 1))([attention, img_value])
            att_img = head_att_img
            concat_key = img_key
        # att_img = Concatenate(axis=1)(att_img)
        # att_img = Dense(512, use_bias=False)(att_img)
        # att_img = Dropout(0.3)(att_img)
        # att_img = Add()([att_img, inpAttImg_query])
        # att_img = LayerNormalization()(att_img)
        # att_img2 = Dense(1024,activation='relu')(att_img)
        # att_img2 = Dense(512)(att_img)
        # att_img2 = Dropout(0.1)(att_img2)
        # att_img = Add()([att_img, att_img2])
        # att_img = LayerNormalization()(att_img)
        att_img2 = Average()([
            Activation('relu')(BatchNormalization()(
                Dense(final_hid)(att_img))),
            Activation('relu')(BatchNormalization()(
                Dense(final_hid)(att_img))),
            Activation('relu')(BatchNormalization()(
                Dense(final_hid)(att_img))),
            Activation('relu')(BatchNormalization()(Dense(final_hid)(att_img)))
        ])
        att_img2 = Dense(final_hid)(att_img2)
        att_img2 = BatchNormalization()(att_img2)
        att_img2 = Activation('relu')(att_img2)
        att_img2 = Dropout(drop_rate)(att_img2)
        att_img = Add()([inpAttImg_query, att_img2])
        att_img = LayerNormalization()(att_img)

        # concat_key = Concatenate(axis=1)(concat_key)
        # concat_key = Dense(512)(concat_key)
        # concat_key = Dropout(0.3)(inpAttImg_key)
        # concat_key = Add()([concat_key, inpAttImg_key])
        # concat_key = LayerNormalization()(concat_key)
        # concat_key2 = Dense(1024,activation='relu')(concat_key)
        # concat_key2 = Dense(512)(concat_key)
        # concat_key2 = Dropout(0.1)(concat_key2)
        # concat_key = Add()([concat_key, concat_key2])
        # concat_key = LayerNormalization()(concat_key)

        inpAttImg_query = att_img
        inpAttImg_key = concat_key

    # merge.append(att_img)

    ##attention image on text
    inpAttText_key = l2_pool
    inpAttText_query = image_append
    for layer in range(1):
        att_text = []
        concat_key = []
        for _ in range(head):
            img_query = Dense(att_hid / head, use_bias=False)(inpAttText_query)
            #     # # img_query = BatchNormalization()(img_query)
            #     # # img_query = Activation('tanh')(img_query)
            text_key = Dense(att_hid / head, use_bias=False)(inpAttText_key)
            #     # # text_key = BatchNormalization()(text_key)
            #     # # text_key = Activation('tanh')(text_key)
            text_value = Dense(att_hid / head, use_bias=False)(inpAttText_key)
            attention = Dot(axes=(1, 2))([img_query, text_key])
            attention = Lambda(lambda x: x[0] / x[1])(
                [attention, np.sqrt(att_hid / head)])
            attention = Activation("softmax")(attention)
            head_att_text = Dot(axes=(1, 1))([attention, text_value])
            att_text = head_att_text
            concat_key = text_key
        # att_text = Concatenate(axis=1)(att_text)
        # att_text = Dense(512, use_bias=False)(att_text)
        # att_text = Dropout(0.3)(att_text)
        # att_text = Add()([att_text, inpAttText_query])
        # att_text = LayerNormalization()(att_text)
        # att_text2 = Dense(1024,activation='relu')(att_text)
        # att_text2 = Dense(512)(att_text)
        # att_text2 = Dropout(0.1)(att_text2)
        # att_text = Add()([att_text, att_text2])
        # att_text = LayerNormalization()(att_text)

        # concat_key = Concatenate(axis=1)(concat_key)
        # concat_key = Dense(512)(concat_key)
        # concat_key = Dropout(0.3)(concat_key)
        # concat_key = Add()([concat_key, inpAttText_key])
        # concat_key = LayerNormalization()(concat_key)
        # concat_key2 = Dense(1024,activation='relu')(concat_key)
        # concat_key2 = Dense(512)(concat_key)
        # concat_key2 = Dropout(0.1)(concat_key2)
        # concat_key = Add()([concat_key, concat_key2])
        # concat_key = LayerNormalization()(concat_key)
        att_text2 = Average()([
            Activation('relu')(BatchNormalization()(
                Dense(final_hid)(att_text))),
            Activation('relu')(BatchNormalization()(
                Dense(final_hid)(att_text))),
            Activation('relu')(BatchNormalization()(
                Dense(final_hid)(att_text))),
            Activation('relu')(BatchNormalization()(
                Dense(final_hid)(att_text)))
        ])
        att_text2 = Dense(final_hid)(att_text2)
        att_text2 = BatchNormalization()(att_text2)
        att_text2 = Activation('relu')(att_text2)
        att_text2 = Dropout(drop_rate)(att_text2)
        att_text = Add()([inpAttText_query, att_text2])
        att_text = LayerNormalization()(att_text)

        inpAttText_query = att_text
        inpAttText_key = concat_key

    head = 1
    ###Self attention image
    inpAttSelf_key = img
    inpAttSelf_query = img
    for layer in range(att_layers):
        self_img = []
        for _ in range(head):
            self_query = Dense(att_hid / head,
                               use_bias=False)(inpAttSelf_query)
            self_key = Dense(att_hid / head, use_bias=False)(inpAttSelf_key)
            self_value = Dense(att_hid / head, use_bias=False)(inpAttSelf_key)

            attention = Dot(axes=(2, 2))([self_query, self_key])
            attention = Lambda(lambda x: x[0] / x[1])(
                [attention, np.sqrt(att_hid / head)])
            attention = Activation("softmax")(attention)
            head_att_self = Dot(axes=(2, 1))([attention, self_value])
            self_img = head_att_self

        # self_img = Concatenate(axis=2)(self_img)
        # self_img = Dense(512, use_bias=False)(self_img)
        # self_img = Dropout(0.3)(self_img)
        # self_img = Add()([self_img, inpAttSelf_query])
        # self_img = LayerNormalization()(self_img)
        # self_img2 = Dense(1024,activation='relu')(self_img)
        # self_img2 = Dense(512)(self_img2)
        # self_img2 = Dropout(0.3)(self_img2)
        # self_img = Add()([self_img, self_img2])
        # self_img = LayerNormalization()(self_img)
        self_img2 = Average()([
            Activation('relu')(BatchNormalization()(
                Dense(final_hid)(self_img))),
            Activation('relu')(BatchNormalization()(
                Dense(final_hid)(self_img))),
            Activation('relu')(BatchNormalization()(
                Dense(final_hid)(self_img))),
            Activation('relu')(BatchNormalization()(
                Dense(final_hid)(self_img)))
        ])
        self_img2 = Dense(final_hid)(self_img2)
        self_img2 = BatchNormalization()(self_img2)
        self_img2 = Activation('relu')(self_img2)
        self_img2 = Dropout(drop_rate)(self_img2)
        self_img = Add()([inpAttSelf_query, self_img2])
        self_img = LayerNormalization()(self_img)

        inpAttSelf_query = self_img
        inpAttSelf_key = self_img

    # merge.append(att_text)
    self_img = Flatten()(self_img)
    # att_img =  Dense(32)(Flatten()(att_img))
    # # # att_img = BatchNormalization()(att_img)
    # att_img = Activation('relu')(att_img)

    # att_text = Flatten()(att_text)
    # # # att_text = Concatenate(axis=1)([att_text,self_img])
    # att_text = Dense(32)(att_text)
    # # # att_text = BatchNormalization()(att_text)
    # att_text = Activation('relu')(att_text)

    self_img = Dense(final_hid)(self_img)
    self_img = BatchNormalization()(self_img)
    self_img = Activation('relu')(self_img)

    # self_img =  Dense(32)(self_img)
    # # # self_img = BatchNormalization()(self_img)
    # self_img = Activation('relu')(self_img)

    # text_append =  Dense(512)(Flatten()(text_append))
    # text_append = BatchNormalization()(text_append)
    # text_append = Activation('relu')(text_append)

    # image_append =  Dense(512)(Flatten()(image_append))
    # image_append = BatchNormalization()(image_append)
    # image_append = Activation('relu')(image_append)

    # merge.append(att_img)
    # merge.append(att_text)
    merge.append(self_img)

    # merge.append(Dense(512,activation='relu')(Flatten()(self_img)))
    # merge.append(text_append)
    merge.append(image_append)
    l_merge = Concatenate(axis=1)(merge)
    # l_merge = image_append
    l_merge = Dropout(drop_rate)(l_merge)
    l_merge = Dense(final_hid)(l_merge)
    l_merge = BatchNormalization()(l_merge)
    l_merge = Activation('relu')(l_merge)
    l_merge = Dropout(drop_rate)(l_merge)
    # l_merge             = Average()(merge)
    out = Dense(N_LABELS, activation='sigmoid')(l_merge)
    model = Model(inputs=[input_ids, attention_mask, image.input], outputs=out)
    
    return model

In [26]:
%%time
EPOCHS          = 8
total_steps     = len(y_train) * BATCH_SIZE
train_data_size = len(y_train)
steps_per_epoch = int(train_data_size / BATCH_SIZE) + 1
num_train_steps = steps_per_epoch * EPOCHS
# warmup_steps    = int(steps_per_epoch * 1)
warmup_steps    = 0

# Create the learning rate scheduler.
decay_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
      initial_learning_rate=1e-4,
      decay_steps=num_train_steps,
      end_learning_rate=0)

warmup_schedule = nlp.optimization.WarmUp(
        initial_learning_rate=7e-5,
        decay_schedule_fn=decay_schedule,
        warmup_steps=warmup_steps)

optimizer       = nlp.optimization.AdamWeightDecay(
        learning_rate=warmup_schedule,
        epsilon=1e-8,
        exclude_from_weight_decay=['LayerNorm', 'layer_norm', 'bias'])

#Load bert4news
# config = AutoConfig.from_pretrained('/content/drive/MyDrive/bert4news/config.json')
# transformer = TFAutoModel.from_pretrained('/content/drive/MyDrive/bert4news/pytorch_model.bin', from_pt=True, config=config)

transformer = TFAutoModel.from_pretrained(MODEL,output_attentions=False,output_hidden_states=True,return_dict =True)
transformer.trainable = False
model = create_model(transformer, max_len=MAX_LENGTH)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                loss='binary_crossentropy',
                metrics='accuracy')

model.summary()

Some layers from the model checkpoint at vinai/bertweet-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/bertweet-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 lambda (Lambda)                (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['lambda[0][0]']                 
                                )                                                                 
                                                                                            

                                                                                                  
 dense_26 (Dense)               (None, 49, 32)       1056        ['dot_5[0][0]']                  
                                                                                                  
 dense_27 (Dense)               (None, 49, 32)       1056        ['dot_5[0][0]']                  
                                                                                                  
 batch_normalization_21 (BatchN  (None, 49, 32)      128         ['dense_24[0][0]']               
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_22 (BatchN  (None, 49, 32)      128         ['dense_25[0][0]']               
 ormalization)                                                                                    
          

 batch_normalization_27 (BatchN  (None, 32)          128         ['dense_30[0][0]']               
 ormalization)                                                                                    
                                                                                                  
 activation_30 (Activation)     (None, 32)           0           ['batch_normalization_27[0][0]'] 
                                                                                                  
 dropout_43 (Dropout)           (None, 32)           0           ['activation_30[0][0]']          
                                                                                                  
 input_ids (InputLayer)         [(None, 32)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 32)]         0           []                               
          

In [27]:
from sklearn.utils import class_weight

# class_weights = class_weight_weight.compute_class_weight('balanced', np.unique(np.argmax(train_labels, axis=1)), np.argmax(train_labels, axis=1))
class_weights = class_weight.compute_class_weight('balanced', classes=[0, 1], y=train_labels)
class_weights = {i : class_weights[i] for i in range(2)}
class_weights

{0: 0.6902455507997297, 1: 1.8140911782119598}

## Train model

In [28]:
from keras.utils import np_utils
from keras.callbacks import Callback, EarlyStopping
from sklearn.metrics import f1_score
from sklearn.datasets import make_classification

class roc_auc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]


    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        # y_pred_train = self.model.predict(self.x, verbose=0)
        # y_pred_train = (y_pred_train>=0.5)*1
        # y_true_train = self.y
        # f1_train_micro = f1_score(y_true_train, y_pred_train, average='micro')
        # f1_train_macro = f1_score(y_true_train, y_pred_train, average='macro')
        
        y_pred_val = self.model.predict(self.x_val, verbose=0)
        y_pred_val = (y_pred_val>0.5)*1
        y_true_val = self.y_val
        f1_val_micro = f1_score(y_true_val, y_pred_val, average='micro')
        f1_val_macro = f1_score(y_true_val, y_pred_val, average='macro')
        print('\ f1_val: micro - %s macro - %s' % (str(round(f1_val_micro,4)),str(round(f1_val_macro,4))),end=100*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return


In [ ]:
n_steps = int(np.ceil(y_train.shape[0] / BATCH_SIZE))

# Checkpoint path
ckpt_path     = f'/Users/mryu/Desktop/{MODEL_NAME}/'
if not os.path.exists(ckpt_path):
    os.makedirs(ckpt_path)
ckpt_path     += 'cp-{epoch:02d}.h5'

# Callback
my_callbacks  = [tf.keras.callbacks.ModelCheckpoint(filepath=ckpt_path, 
                                                    monitor='val_loss', 
                                                    save_weights_only=True,
                                                    save_freq='epoch'),
                 tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1),
                 roc_auc_callback(training_data=(X_train, y_train),validation_data=(X_val, y_val))]

H = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    #steps_per_epoch=n_steps,
    # class_weight=class_weights,
    shuffle=False,
    callbacks=my_callbacks
)

Epoch 1/8


# Saving Fine-Tuned Model

In [ ]:
model.save_weights(os.path.join(base_dir, f'model/{MODEL_NAME}.h5'))

In [ ]:
EPOCHS = 20

In [ ]:
%%time
EPOCHS          = EPOCHS
total_steps     = len(y_train) * BATCH_SIZE
train_data_size = len(y_train)
steps_per_epoch = int(train_data_size / BATCH_SIZE) + 1
num_train_steps = steps_per_epoch * EPOCHS
# warmup_steps    = int(num_train_steps * 0.1)
warmup_steps    = 0

# Create the learning rate scheduler.
decay_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
      initial_learning_rate=2e-5,
      decay_steps=num_train_steps,
      end_learning_rate=0)

warmup_schedule = nlp.optimization.WarmUp(
        initial_learning_rate=2e-5,
        decay_schedule_fn=decay_schedule,
        warmup_steps=warmup_steps)

optimizer       = nlp.optimization.AdamWeightDecay(
        learning_rate=warmup_schedule,
        epsilon=1e-8,
        exclude_from_weight_decay=['LayerNorm', 'layer_norm', 'bias'])


#Load bert4news
# config = AutoConfig.from_pretrained('/content/drive/MyDrive/bert4news/config.json')
# transformer = TFAutoModel.from_pretrained('/content/drive/MyDrive/bert4news/pytorch_model.bin', from_pt=True, config=config)

transformer = TFAutoModel.from_pretrained(MODEL,output_attentions=False,output_hidden_states=True,return_dict =True)
transformer.trainable = False
model2      = create_model(transformer, max_len=MAX_LENGTH)
model2.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics='accuracy')
    
model2.summary()

In [ ]:
model2.load_weights(os.path.join(base_dir, f'checkpoint-test/{MODEL_NAME}/cp-03.h5'))
# model2.evaluate(X_val,y_val)

 # Predict

In [ ]:
input_ids       = []
attention_masks = []

for sent in tqdm(test_sent):
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, 
                        max_length = MAX_LENGTH,         
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'np',   
                        truncation = True,
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])


id_test_data        = np.concatenate(input_ids)
mask_test_data      = np.concatenate(attention_masks)
id_test_data.shape, mask_test_data.shape

In [ ]:
pred = model2.predict(X_test_data, verbose=1)
test_true = test_labels
test_true.shape, pred.shape

In [33]:
X_test_data         = [
    id_test_data,
    mask_test_data,
    img_test
]

NameError: name 'id_test_data' is not defined

In [ ]:
from sklearn import metrics
theta = 0.4
# Print the confusion matrix
print(metrics.confusion_matrix(test_true,  (pred>theta).astype(int)))

# Print the precision and recall, among other metrics

print(metrics.classification_report(test_true, (pred>theta).astype(int), digits=3))
